# 1. Setting Up the Environment

## 1.1. Install dependencies:
This command updates the system's package list and installs various libraries required for running Chrome and Selenium.

In [1]:
!apt-get update -y
!apt-get install -y \
libglib2.0-0 \
libnss3 \
libdbus-glib-1-2 \
libgconf-2-4 \
libfontconfig1 \
libvulkan1 \
gconf2-common \
libwayland-server0 \
libgbm1 \
udev \
libu2f-udev 
!apt --fix-broken install -y  

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [128 kB]
Get:2 http://packages.cloud.google.com/apt gcsfuse-focal InRelease [1227 B]
Get:3 https://packages.cloud.google.com/apt cloud-sdk InRelease [1618 B]
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Get:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease [128 kB]
Get:6 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [3933 kB]
Get:7 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1258 kB]
Get:8 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [3926 kB]
Get:9 http://security.ubuntu.com/ubuntu focal-security/multiverse amd64 Packages [30.9 kB]
Get:10 http://archive.ubuntu.com/ubuntu focal-backports InRelease [128 kB]
Get:11 https://packages.cloud.google.com/apt cloud-sdk/main amd64 Packages [3214 kB]
Get:12 https://packages.cloud.google.com/apt cloud-sdk/main all Packages [1511 kB]
Get:13 http://archive.ubuntu.com/ubuntu focal-updat

## 1.2. Download and extract Chrome:

To use Selenium, you will need to download and install Chrome and Chromedriver.

* **Chrome**: Chrome is a popular web browser that is known for its speed and security.
* **Chromedriver**: Chromedriver is a tool that allows Selenium to interact with Chrome.

Downloads the latest stable version of Chrome for Linux and extracts it to the /usr/bin directory.

In [2]:
!wget -P /tmp https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/116.0.5845.96/linux64/chrome-linux64.zip
!unzip /tmp/chrome-linux64.zip -d /usr/bin/

--2024-08-27 21:59:14--  https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/116.0.5845.96/linux64/chrome-linux64.zip
Resolving edgedl.me.gvt1.com (edgedl.me.gvt1.com)... 34.104.35.123, 2600:1900:4110:86f::
Connecting to edgedl.me.gvt1.com (edgedl.me.gvt1.com)|34.104.35.123|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://storage.googleapis.com/chrome-for-testing-public/116.0.5845.96/linux64/chrome-linux64.zip [following]
--2024-08-27 21:59:15--  https://storage.googleapis.com/chrome-for-testing-public/116.0.5845.96/linux64/chrome-linux64.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.119.207, 142.251.31.207, 74.125.143.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.119.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 145898081 (139M) [application/zip]
Saving to: '/tmp/chrome-linux64.zip'

chrome-linux64.zip  100%[===================>] 139.14M  44.6MB/s  

## 1.3. Download and extract Chromedriver:

As it was done in the previous code.

In [3]:
!wget -P /tmp https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/116.0.5845.96/linux64/chromedriver-linux64.zip
!unzip /tmp/chromedriver-linux64.zip -d /usr/bin/

--2024-08-27 21:59:25--  https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/116.0.5845.96/linux64/chromedriver-linux64.zip
Resolving edgedl.me.gvt1.com (edgedl.me.gvt1.com)... 34.104.35.123, 2600:1900:4110:86f::
Connecting to edgedl.me.gvt1.com (edgedl.me.gvt1.com)|34.104.35.123|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://storage.googleapis.com/chrome-for-testing-public/116.0.5845.96/linux64/chromedriver-linux64.zip [following]
--2024-08-27 21:59:25--  https://storage.googleapis.com/chrome-for-testing-public/116.0.5845.96/linux64/chromedriver-linux64.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.153.207, 142.251.18.207, 108.177.127.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.153.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7271942 (6.9M) [application/zip]
Saving to: '/tmp/chromedriver-linux64.zip'

chromedriver-linux6 100%[==================

## 1.4. Install Python libraries

In [4]:
!apt install -y python3-selenium
!pip install selenium==3.141.0




The following additional packages will be installed:
  apparmor chromium-browser chromium-chromedriver liblzo2-2 snapd
  squashfs-tools
Suggested packages:
  apparmor-profiles-extra apparmor-utils zenity | kdialog
The following NEW packages will be installed:
  apparmor chromium-browser chromium-chromedriver liblzo2-2 python3-selenium
  snapd squashfs-tools
0 upgraded, 7 newly installed, 0 to remove and 128 not upgraded.
Need to get 25.9 MB of archives.
After this operation, 107 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 apparmor amd64 2.13.3-7ubuntu5.3 [502 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/main amd64 liblzo2-2 amd64 2.10-2 [50.8 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 squashfs-tools amd64 1:4.4-1ubuntu0.3 [117 kB]
Get:4 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 snapd amd64 2.63+20.04ubuntu0.1 [25.1 MB]
Get:5 http://archive.ubuntu.com/ubuntu focal-updates/univer

# 2.Importing Libraries

You will also need to install the following Python libraries:

* **selenium**: The Selenium library provides the API for interacting with web pages.
* **webdriver**: The webdriver library provides a way to interact with web drivers, such as Chromedriver.
* **BeautifulSoup**: The BeautifulSoup library is used for parsing HTML content.

In [5]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

In [6]:
from retrying import retry
import time
import traceback

# 3. Configuring Chrome Driver

These functions define the locations of Chrome and Chromedriver executables. Additionally, initialize_driver creates a Chrome webdriver instance with specific options:

* *--headless*: Runs Chrome in headless mode, making it invisible.
* *--no-sandbox*: Disables the sandbox for improved performance.
* *--start-fullscreen*: Starts Chrome in fullscreen mode.
* *--allow-insecure-localhost*: Allows access to insecure local websites (if needed).
* *--disable-dev-shm-usage*: Disables shared memory usage for Chrome.
* *user-agent*: Sets the user agent string to mimic a regular browser.

In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

CHROME_BINARY_LOCATION = "/usr/bin/chrome-linux64/chrome"
CHROMEDRIVER_BINARY_LOCATION = "/usr/bin/chromedriver-linux64/chromedriver"

def add_driver_options(options):
    """
    Add configurable options
    """
    chrome_options = Options()
    for opt in options:
        chrome_options.add_argument(opt)
    return chrome_options

def initialize_driver():
    """
    Initialize the web driver
    """
    driver_config = {
        "options": [
            "--headless",
            "--no-sandbox",
            "--start-fullscreen",
            "--allow-insecure-localhost",
            "--disable-dev-shm-usage",
            "user-agent=Chrome/116.0.5845.96"
        ],
    }
    options = add_driver_options(driver_config["options"])
    options.binary_location = CHROME_BINARY_LOCATION
    driver = webdriver.Chrome(
        executable_path=CHROMEDRIVER_BINARY_LOCATION,
        options=options)
    return driver


# 4. Using Selenium to scrape movie_urls and reviews

In [8]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from tqdm import tqdm
import pandas as pd

# Define the WebDriver initialization function
def initialize_driver():
    CHROME_BINARY_LOCATION = "/usr/bin/chrome-linux64/chrome"
    CHROMEDRIVER_BINARY_LOCATION = "/usr/bin/chromedriver-linux64/chromedriver"

    chrome_options = Options()
    chrome_options.binary_location = CHROME_BINARY_LOCATION
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--start-fullscreen")
    chrome_options.add_argument("--allow-insecure-localhost")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("user-agent=Chrome/116.0.5845.96")

    driver = webdriver.Chrome(
        executable_path=CHROMEDRIVER_BINARY_LOCATION,
        options=chrome_options
    )
    return driver

# Define the function to scrape movie URLs
def scrape_movie_urls(driver, num_movies):
    """
    Scrape a number of popular movie URLs from Letterboxd.

    Args:
        driver (webdriver.Chrome): The initialized Chrome driver.
        num_movies (int): Number of movie URLs to scrape.

    Returns:
        pd.DataFrame: DataFrame containing the movie URLs.
    """
    print("...Connected to selenium service!")
    movie_urls = []
    page = 1

    try:
        while len(movie_urls) < num_movies:
            driver.get(f"https://letterboxd.com/films/popular/page/{page}/")
            
            # Wait for the movie elements to be present
            WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'a.frame'))
            )
            
            elements = driver.find_elements(By.CSS_SELECTOR, 'a.frame')
            for element in tqdm(elements, desc=f"Processing Page {page}", leave=False):
                if len(movie_urls) >= num_movies:
                    break
                movie_urls.append(element.get_attribute('href'))
            
            print(f"Page {page} processed, {len(movie_urls)} URLs found.")
            page += 1
        
        df = pd.DataFrame(movie_urls, columns=['url'])
        print("Movie URLs successfully scraped!\nMovie URLs preview:")
        print(df.head().to_string())
    
    except Exception as e:
        print(f"An error occurred: {e}")
    
    finally:
        print("\nClosing connection to selenium webdriver...")
        driver.quit()
        print("Connection closed successfully!\n\n")
    
    return df




In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from tqdm import tqdm
import pandas as pd
import os

# Initialize the WebDriver
def initialize_driver():
    CHROME_BINARY_LOCATION = "/usr/bin/chrome-linux64/chrome"
    CHROMEDRIVER_BINARY_LOCATION = "/usr/bin/chromedriver-linux64/chromedriver"

    chrome_options = Options()
    chrome_options.binary_location = CHROME_BINARY_LOCATION
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--start-fullscreen")
    chrome_options.add_argument("--allow-insecure-localhost")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("user-agent=Chrome/116.0.5845.96")

    driver = webdriver.Chrome(
        executable_path=CHROMEDRIVER_BINARY_LOCATION,
        options=chrome_options
    )
    return driver

# Normalize the star rating
def normalize_rating(star_rating):
    rating_map = {'★': 0.2, '½': 0.1}
    return sum(rating_map[char] for char in star_rating if char in rating_map)

# Extract review details from a review element
def extract_movie_review_details(element):
    review = {}

    try:
        user_url_element = element.find_element(By.CSS_SELECTOR, 'a.avatar.-a40')
        review['user_url'] = user_url_element.get_attribute('href')
    except NoSuchElementException:
        review['user_url'] = None

    try:
        review_text_element = element.find_element(By.CSS_SELECTOR, 'div.body-text.-prose.collapsible-text')
        review['review_text'] = review_text_element.text
    except NoSuchElementException:
        review['review_text'] = None

    try:
        rating_element = element.find_element(By.CSS_SELECTOR, 'span.rating')
        review['rating'] = normalize_rating(rating_element.text)
    except NoSuchElementException:
        review['rating'] = None

    try:
        review_date_element = element.find_element(By.CSS_SELECTOR, 'span.date > span._nobr')
        review['review_date'] = review_date_element.text
    except NoSuchElementException:
        review['review_date'] = None

    return review

# Scrape reviews for a single movie
def scrape_reviews_for_movie(driver, movie_url, num_reviews_per_movie):
    movie_reviews = []
    page = 1

    try:
        driver.get(f"{movie_url}/reviews/")

        # Scrape movie title and release year
        try:
            movie_title_element = driver.find_element(By.CSS_SELECTOR, 'div.contextual-title h1.headline-2 a')
            movie_title = movie_title_element.text
        except NoSuchElementException:
            movie_title = None

        try:
            movie_year_element = driver.find_element(By.CSS_SELECTOR, 'div.contextual-title h1.headline-2 small.metadata a')
            movie_year = movie_year_element.text
        except NoSuchElementException:
            movie_year = None

        while len(movie_reviews) < num_reviews_per_movie:
            review_elements = driver.find_elements(By.CSS_SELECTOR, 'li.film-detail')

            for element in review_elements:
                if len(movie_reviews) >= num_reviews_per_movie:
                    break

                review = extract_movie_review_details(element)
                review['movie_title'] = movie_title
                review['movie_year'] = movie_year
                review['movie_url'] = movie_url

                if review['review_text'] and review['rating'] is not None:
                    movie_reviews.append(review)

            if len(movie_reviews) < num_reviews_per_movie:
                page += 1
                driver.get(f"{movie_url}/reviews/page/{page}/")
            else:
                break

    except TimeoutException:
        print(f"Timeout while loading reviews for movie {movie_url}")

    return movie_reviews

# Save the DataFrame as a checkpoint
def save_checkpoint(df, checkpoint_filename):
    df.to_csv(checkpoint_filename, index=False)
    print(f"Checkpoint saved to {checkpoint_filename}")

# Scrape reviews from a list of movie URLs
def scrape_reviews_from_movies(driver, movie_urls, num_reviews_per_movie, checkpoint_filename="movie_reviews_checkpoint.csv"):
    print("Successfully connected to WebDriver!")
    all_reviews = []
    checkpoint_interval = 10  # Save every 10 movies

    for idx, movie_url in enumerate(tqdm(movie_urls, desc="Processing Movies"), start=1):
        print(f"Scraping reviews for movie: {movie_url}")
        movie_reviews = scrape_reviews_for_movie(driver, movie_url, num_reviews_per_movie)
        all_reviews.extend(movie_reviews)

        # Save checkpoint at intervals
        if idx % checkpoint_interval == 0:
            df = pd.DataFrame(all_reviews)
            save_checkpoint(df, checkpoint_filename)

    # Save the final data
    df = pd.DataFrame(all_reviews)
    save_checkpoint(df, checkpoint_filename)

    print("Reviews successfully scraped from movie URLs!\nReviews preview:")
    print(df.head().to_string())
    print("\nClosing connection to selenium WebDriver...")
    driver.quit()
    print("Connection closed successfully!\n\n")

    return df




In [10]:
if __name__ == "__main__":
    driver = initialize_driver()
    try:
        num_movies_to_scrape = 200  # Adjust this number as needed
        movie_urls_df = scrape_movie_urls(driver, num_movies_to_scrape)
        movie_urls_df.to_csv('/kaggle/working/movie_urls.csv')
    finally:
        driver.quit()  # Ensure the driver is closed after usage

...Connected to selenium service!


Page 1 processed, 72 URLs found.


Page 2 processed, 144 URLs found.


Page 3 processed, 200 URLs found.
Movie URLs successfully scraped!
Movie URLs preview:
                                                              url
0                             https://letterboxd.com/film/barbie/
1                      https://letterboxd.com/film/parasite-2019/
2  https://letterboxd.com/film/everything-everywhere-all-at-once/
3                         https://letterboxd.com/film/fight-club/
4                         https://letterboxd.com/film/la-la-land/

Closing connection to selenium webdriver...
Connection closed successfully!




In [11]:
if __name__ == "__main__":
    driver = initialize_driver()
    try:
        num_reviews_per_movie = 100  # Adjust as needed
        movie_urls_df = pd.read_csv('movie_urls.csv')  # Assuming you have a CSV file of movie URLs
        reviews_df = scrape_reviews_from_movies(driver, movie_urls_df['url'], num_reviews_per_movie)
        # Save to file if needed
        reviews_df.to_csv('/kaggle/working/movie_reviews.csv')
    finally:
        driver.quit()  # Ensure the driver is closed after usage

Successfully connected to WebDriver!


Processing Movies:   0%|          | 0/200 [00:00<?, ?it/s]

Scraping reviews for movie: https://letterboxd.com/film/barbie/


Processing Movies:   0%|          | 1/200 [01:22<4:32:45, 82.24s/it]

Scraping reviews for movie: https://letterboxd.com/film/parasite-2019/


...

Processing Movies: 100%|█████████▉| 199/200 [4:18:47<01:15, 75.20s/it]

Scraping reviews for movie: https://letterboxd.com/film/the-whale-2022/


Processing Movies: 100%|██████████| 200/200 [4:20:08<00:00, 78.04s/it]

Checkpoint saved to movie_reviews_checkpoint.csv


Checkpoint saved to movie_reviews_checkpoint.csv
Reviews successfully scraped from movie URLs!
Reviews preview:
                              user_url                                                                                                                                                                                                                                                                                                                                                          review_text  rating  review_date movie_title movie_year                            movie_url
0     https://letterboxd.com/dudstard/                                                                                                                                                                                                                                                                                                                                chorei de me acabar vey foi tudo amo💋     1.0  27 Aug